In [2]:
from config import config_erkan, config_halit
from binance.um_futures import UMFutures
import ta
import pandas as pd
from time import sleep
from binance.error import ClientError
import logging
from binance.client import Client
from binance.exceptions import BinanceAPIException
import pandas as pd
import time
from datetime import datetime

In [3]:
client = UMFutures(key = config_erkan['key'], secret=config_erkan['secret'])

In [4]:
tp = 0.012
sl = 0.009
volume = 10  # volume for one order (if its 10 and leverage is 10, then you put 1 usdt to one position)
leverage = 1
type = 'ISOLATED'  # type is 'ISOLATED' or 'CROSS'
qty = 5

In [59]:
# getting your futures balance in USDT
def get_balance_usdt():
    try:
        response = client.balance(recvWindow=6000)
        for elem in response:
            if float(elem['balance']) > 0:
                print(elem['asset'],":", float(elem['balance']))

    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

print(get_balance_usdt())

USDT : 744.48286637
None


In [6]:
def get_tickers_usdt():
    tickers = []
    resp = client.ticker_price()
    for elem in resp:
        if 'USDT' in elem['symbol']:
            tickers.append(elem['symbol'])
    return tickers

# Getting candles for the needed symbol, its a dataframe with 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'
def klines(symbol):
    try:
        resp = pd.DataFrame(client.klines(symbol, '1h'))
        resp = resp.iloc[:,:6]
        resp.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
        resp = resp.set_index('Time')
        resp.index = pd.to_datetime(resp.index, unit = 'ms')
        resp = resp.astype(float)
        return resp
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )


# Set leverage for the needed symbol. You need this bcz different symbols can have different leverage
def set_leverage(symbol, level):
    try:
        response = client.change_leverage(
            symbol=symbol, leverage=level, recvWindow=6000
        )
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

# The same for the margin type
def set_mode(symbol, type):
    try:
        response = client.change_margin_type(
            symbol=symbol, marginType=type, recvWindow=6000
        )
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

# Price precision. BTC has 1, XRP has 4
def get_price_precision(symbol):
    resp = client.exchange_info()['symbols']
    for elem in resp:
        if elem['symbol'] == symbol:
            return elem['pricePrecision']


# Amount precision. BTC has 3, XRP has 1
def get_qty_precision(symbol):
    resp = client.exchange_info()['symbols']
    for elem in resp:
        if elem['symbol'] == symbol:
            return elem['quantityPrecision']
        




In [7]:
data = klines('BTCUSDT')
print(data)

                        Open     High      Low    Close     Volume
Time                                                              
2024-11-04 06:00:00  68811.4  69159.3  68808.1  69115.6   4747.282
2024-11-04 07:00:00  69115.5  69166.0  68600.9  68601.0   6073.573
2024-11-04 08:00:00  68601.0  68748.0  68456.3  68686.5   7645.132
2024-11-04 09:00:00  68686.5  68913.0  68459.8  68628.1   7179.225
2024-11-04 10:00:00  68628.0  68903.0  68587.4  68817.0   5480.654
...                      ...      ...      ...      ...        ...
2024-11-24 21:00:00  96888.0  97061.4  96645.3  97037.7   3708.058
2024-11-24 22:00:00  97037.7  98086.3  97037.7  98082.0  10276.195
2024-11-24 23:00:00  98082.1  98429.7  97577.7  97960.4  11637.611
2024-11-25 00:00:00  97960.5  98106.9  97574.4  97763.2   5664.024
2024-11-25 01:00:00  97763.2  97900.3  96929.9  97225.7   8373.367

[500 rows x 5 columns]


In [38]:
# Your current positions (returns the symbols list):
def get_pos():
    try:
        resp = client.get_position_risk()
        pos = []
        for elem in resp:
            if float(elem['positionAmt']) != 0:
                pos.append(elem['symbol'])
        return pos
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

def check_orders():
    try:
        response = client.get_orders(recvWindow=6000)
        sym = []
        for elem in response:
            sym.append(elem['symbol'])
        return sym
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

# Close open orders for the needed symbol. If one stop order is executed and another one is still there
def close_open_orders(symbol):
    try:
        response = client.cancel_open_orders(symbol=symbol, recvWindow=6000)
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

In [9]:
orders = 0
symbol = ''
# getting all symbols from Binace Futures list:
symbols = get_tickers_usdt()

In [61]:
print(get_balance_usdt())

USDT : 744.33672078
None


In [56]:
pos = get_pos()
len(pos)

0

In [58]:
ord = check_orders()
len(ord)

1

In [13]:
def get_pnl():
    try:
        response = client.get_income_history(incomeType="REALIZED_PNL", recvWindow=6000)
        if response:
            # En son gerçekleşen işlemin kar/zarar bilgisi
            latest_pnl = float(response[0]['income'])
            symbol = response[0]['symbol']
            return symbol, latest_pnl
        return None, None
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )
        return None, None

In [14]:
while True:
    symbol, pnl = get_pnl()
    if symbol and pnl:
        print(f"🔔 {symbol} için gerçekleşen kar/zarar: {pnl} USDT")
    sleep(3)

🔔 ACHUSDT için gerçekleşen kar/zarar: 0.07807 USDT
🔔 ACHUSDT için gerçekleşen kar/zarar: 0.07807 USDT
🔔 ACHUSDT için gerçekleşen kar/zarar: 0.07807 USDT
🔔 ACHUSDT için gerçekleşen kar/zarar: 0.07807 USDT
🔔 ACHUSDT için gerçekleşen kar/zarar: 0.07807 USDT
🔔 ALGOUSDT için gerçekleşen kar/zarar: 0.07667 USDT
🔔 ALGOUSDT için gerçekleşen kar/zarar: 0.07667 USDT
🔔 TOKENUSDT için gerçekleşen kar/zarar: 0.0783 USDT
🔔 IOTAUSDT için gerçekleşen kar/zarar: 0.08064 USDT
🔔 STEEMUSDT için gerçekleşen kar/zarar: 0.092 USDT
🔔 STEEMUSDT için gerçekleşen kar/zarar: 0.092 USDT
🔔 STEEMUSDT için gerçekleşen kar/zarar: 0.092 USDT
🔔 STEEMUSDT için gerçekleşen kar/zarar: 0.092 USDT
🔔 ADAUSDT için gerçekleşen kar/zarar: -0.09 USDT
🔔 ADAUSDT için gerçekleşen kar/zarar: -0.09 USDT
🔔 ADAUSDT için gerçekleşen kar/zarar: -0.09 USDT
🔔 ADAUSDT için gerçekleşen kar/zarar: -0.09 USDT
🔔 ADAUSDT için gerçekleşen kar/zarar: -0.09 USDT
🔔 ADAUSDT için gerçekleşen kar/zarar: -0.09 USDT
🔔 ADAUSDT için gerçekleşen kar/zarar: -

KeyboardInterrupt: 

In [52]:
from typing import List, Dict, Optional, Any
from decimal import Decimal
import time
from dataclasses import dataclass
import logging
from pathlib import Path

import pandas as pd
from binance.um_futures import UMFutures
from binance.error import ClientError
import ta

from config import config_erkan
from lstm_predictor import LSTMPredictor

# Setup logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(Path('future_trading.log')),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class TradingConfig:
    """Configuration for trading parameters"""
    volume: float = 10.0
    tp_percentage: float = 0.009  # Take profit percentage
    sl_percentage: float = 0.009  # Stop loss percentage
    retry_attempts: int = 3
    retry_delay: float = 1.0
    leverage: int = 1  # Default leverage
    margin_type: str = "ISOLATED"  # ISOLATED or CROSSED

class BinanceFuturesTrader:
    """Handles all futures trading operations with Binance"""
    
    def __init__(self, api_key: str, api_secret: str, config: TradingConfig = TradingConfig()):
        """Initialize the trader with API credentials and configuration"""
        self.client = UMFutures(key=api_key, secret=api_secret)
        self.config = config
        
    def get_balance_usdt(self) -> Optional[float]:
        """Get USDT balance from futures wallet"""
        for attempt in range(self.config.retry_attempts):
            try:
                response = self.client.balance(recvWindow=6000)
                return next(
                    (float(bal['balance']) for bal in response if bal['asset'] == 'USDT'),
                    None
                )
            except ClientError as error:
                logger.error(f"Attempt {attempt + 1} failed: {error}")
                if attempt == self.config.retry_attempts - 1:
                    raise
                time.sleep(self.config.retry_delay)
    
    def close_all_positions(self) -> None:
        """Close all open positions and cancel their orders"""
        try:
            # Get all open positions
            open_positions = self.get_positions()
            if not open_positions:
                logger.info("No open positions to close")
                return

            logger.info(f"Closing {len(open_positions)} open positions")
            
            # Close each position
            for symbol in open_positions:
                try:
                    # Get position information
                    position_info = self.get_position_pnl(symbol)
                    if not position_info:
                        continue

                    position_size = position_info['position_size']
                    if position_size == 0:
                        continue

                    # Determine side for closing (opposite of current position)
                    side = 'SELL' if position_size > 0 else 'BUY'
                    
                    # Cancel any existing orders first
                    self.close_open_orders(symbol)
                    
                    # Place market order to close position
                    qty = abs(position_size)
                    self.client.new_order(
                        symbol=symbol,
                        side=side,
                        type='MARKET',
                        quantity=qty,
                        reduceOnly=True  # Ensure we only close existing position
                    )
                    
                    logger.info(f"Closed position for {symbol}: {qty} @ market price")
                    
                except ClientError as e:
                    logger.error(f"Failed to close position for {symbol}: {e}")
                    continue
                
            logger.info("Finished closing all positions")
            
        except ClientError as error:
            logger.error(f"Failed to close all positions: {error}")
            raise

    def get_tickers_usdt(self) -> List[str]:
        """Get all available USDT trading pairs"""
        try:
            resp = self.client.ticker_price()
            return [elem['symbol'] for elem in resp if 'USDT' in elem['symbol']]
        except ClientError as error:
            logger.error(f"Failed to get tickers: {error}")
            raise

    def get_historical_klines(self, symbol: str, timeframe: str = '1h') -> pd.DataFrame:
        """Get historical kline data for a symbol"""
        try:
            resp = pd.DataFrame(self.client.klines(symbol, timeframe))
            if resp.empty:
                return pd.DataFrame()
            
            resp = resp.iloc[:, :6]
            resp.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
            resp = resp.set_index('Time')
            resp.index = pd.to_datetime(resp.index, unit='ms')
            return resp.astype(float)
        except ClientError as error:
            logger.error(f"Failed to get klines for {symbol}: {error}")
            raise

    def get_symbol_info(self, symbol: str) -> Dict[str, int]:
        """Get price and quantity precision for a symbol"""
        try:
            symbol_info = next(
                (item for item in self.client.exchange_info()['symbols'] 
                 if item['symbol'] == symbol),
                None
            )
            if not symbol_info:
                raise ValueError(f"Symbol {symbol} not found")
                
            return {
                'price_precision': symbol_info['pricePrecision'],
                'qty_precision': symbol_info['quantityPrecision']
            }
        except ClientError as error:
            logger.error(f"Failed to get symbol info for {symbol}: {error}")
            raise

    def _set_leverage_and_margin(self, symbol: str) -> None:
        """Set leverage and margin type for a symbol"""
        try:
            # Set margin type
            try:
                self.client.change_margin_type(
                    symbol=symbol,
                    marginType=self.config.margin_type,
                    recvWindow=6000
                )
            except ClientError as e:
                # Ignore error if margin type is already set
                if e.error_code != -4046:  # -4046 is "No need to change margin type"
                    raise

            # Set leverage
            self.client.change_leverage(
                symbol=symbol,
                leverage=self.config.leverage,
                recvWindow=6000
            )
            logger.info(f"Set leverage to {self.config.leverage}x and margin type to {self.config.margin_type} for {symbol}")
        except ClientError as error:
            logger.error(f"Failed to set leverage/margin for {symbol}: {error}")
            raise

    def place_order(self, symbol: str, side: str) -> None:
        """Place a new order with stop loss and take profit"""
        try:
            # Log position PnL before placing new order
            current_pnl = self.get_position_pnl(symbol)
            if current_pnl:
                logger.info(f"Current position PnL for {symbol}: {current_pnl['unrealized_pnl']:.2f} USDT ({current_pnl['pnl_percentage']:.2f}%)")
            
            # Set leverage and margin type before placing order
            self._set_leverage_and_margin(symbol)
            
            current_price = float(self.client.ticker_price(symbol)['price'])
            symbol_info = self.get_symbol_info(symbol)
            
            qty = round(
                self.config.volume / current_price,
                symbol_info['qty_precision']
            )
            
            # Place main order
            main_order = self._place_market_order(symbol, side, qty)
            logger.info(f"{symbol} {side} order placed")
            
            # Place stop loss and take profit orders
            self._place_sl_tp_orders(
                symbol, side, qty, current_price,
                symbol_info['price_precision']
            )
        except ClientError as error:
            logger.error(f"Failed to place orders for {symbol}")
            raise

    def _place_market_order(self, symbol: str, side: str, qty: float) -> Dict[str, Any]:
        """Place a market order"""
        return self.client.new_order(
            symbol=symbol,
            side=side.upper(),
            type='MARKET',
            quantity=qty
        )

    def _place_sl_tp_orders(self, symbol: str, side: str, qty: float,
                           current_price: float, price_precision: int) -> None:
        """Place stop loss and take profit orders"""
        is_buy = side.lower() == 'buy'
        sl_price = round(
            current_price * (1 - self.config.sl_percentage if is_buy else 1 + self.config.sl_percentage),
            price_precision
        )
        tp_price = round(
            current_price * (1 + self.config.tp_percentage if is_buy else 1 - self.config.tp_percentage),
            price_precision
        )

        # Place stop loss
        self.client.new_order(
            symbol=symbol,
            side='SELL' if is_buy else 'BUY',
            type='STOP_MARKET',
            quantity=qty,
            timeInForce='GTC',
            stopPrice=sl_price
        )
        
        # Place take profit
        self.client.new_order(
            symbol=symbol,
            side='SELL' if is_buy else 'BUY',
            type='TAKE_PROFIT_MARKET',
            quantity=qty,
            timeInForce='GTC',
            stopPrice=tp_price
        )

    def get_positions(self) -> List[str]:
        """Get current open positions"""
        try:
            positions = self.client.get_position_risk()
            return [
                pos['symbol'] for pos in positions
                if float(pos['positionAmt']) != 0
            ]
        except ClientError as error:
            logger.error(f"Failed to get positions: {error}")
            raise

    def get_open_orders(self) -> List[str]:
        """Get symbols with open orders"""
        try:
            orders = self.client.get_orders(recvWindow=6000)
            return list({order['symbol'] for order in orders})
        except ClientError as error:
            logger.error(f"Failed to get open orders: {error}")
            raise

    def close_open_orders(self, symbol: str) -> None:
        """Cancel all open orders for a symbol"""
        try:
            response = self.client.cancel_open_orders(symbol=symbol, recvWindow=6000)
            print(f"Cancelled open orders for {symbol}: {response}")
        except ClientError as error:
            print(f"Failed to close orders for {symbol}: {error}")
            raise

    def get_position_pnl(self, symbol: str) -> Dict[str, float]:
        """Get PnL information for a specific position"""
        try:
            positions = self.client.get_position_risk(symbol=symbol, recvWindow=6000)
            for position in positions:
                if position['symbol'] == symbol:
                    entry_price = float(position['entryPrice'])
                    mark_price = float(position['markPrice'])
                    position_amt = float(position['positionAmt'])
                    unrealized_pnl = float(position['unRealizedProfit'])
                    
                    return {
                        'symbol': symbol,
                        'entry_price': entry_price,
                        'current_price': mark_price,
                        'position_size': position_amt,
                        'unrealized_pnl': unrealized_pnl,
                        'pnl_percentage': (unrealized_pnl / (abs(position_amt) * entry_price)) * 100 if position_amt != 0 else 0
                    }
            return None
        except ClientError as error:
            logger.error(f"Failed to get PnL for {symbol}: {error}")
            raise

    def get_all_positions_pnl(self) -> List[Dict[str, float]]:
        """Get PnL information for all open positions"""
        try:
            positions = self.client.get_position_risk(recvWindow=6000)
            pnl_info = []
            total_unrealized_pnl = 0
            
            for position in positions:
                position_amt = float(position['positionAmt'])
                if position_amt != 0:  # Only include active positions
                    entry_price = float(position['entryPrice'])
                    mark_price = float(position['markPrice'])
                    unrealized_pnl = float(position['unRealizedProfit'])
                    total_unrealized_pnl += unrealized_pnl
                    
                    pnl_info.append({
                        'symbol': position['symbol'],
                        'entry_price': entry_price,
                        'current_price': mark_price,
                        'position_size': position_amt,
                        'unrealized_pnl': unrealized_pnl,
                        'pnl_percentage': (unrealized_pnl / (abs(position_amt) * entry_price)) * 100
                    })
            
            logger.info(f"Total unrealized PnL: {total_unrealized_pnl:.2f} USDT")
            return pnl_info
        except ClientError as error:
            logger.error(f"Failed to get all positions PnL: {error}")
            raise

    def get_income_history(self, income_type: str = "REALIZED_PNL", limit: int = 100) -> List[Dict]:
        """Get trading income history
        
        Args:
            income_type: Type of income to query (REALIZED_PNL, COMMISSION, FUNDING_FEE)
            limit: Number of records to return
        """
        try:
            income = self.client.get_income_history(
                incomeType=income_type,
                limit=limit,
                recvWindow=6000
            )
            
            total_pnl = sum(float(record['income']) for record in income)
            logger.info(f"Total {income_type}: {total_pnl:.2f} USDT (last {limit} trades)")
            
            return income
        except ClientError as error:
            logger.error(f"Failed to get income history: {error}")
            raise

def main():
    """Main trading loop"""
    trader = BinanceFuturesTrader(
        api_key=config_erkan['key'],
        api_secret=config_erkan['secret']
    )

    trader.close_all_positions()
    
    current_positions = trader.get_positions()
    current_orders = trader.get_open_orders()
    print(len(current_positions))
    # Process existing positions
    for symbol in current_orders:
        try:
            trader.close_open_orders(symbol)
            print(f"closed {symbol}")
        except Exception as e:
            logger.error(f"Error processing position for {symbol}: {e}")
            continue
           
        

main()

Cancelled open orders for EIGENUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for PNUTUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for XMRUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for STORJUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for HMSTRUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for XLMUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for OMNIUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for BNXUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for GLMUSDT: {'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Cancelled open orders for COS